In [1]:
import torch
import numpy as np
import torch.nn as nn
import math
from copy import deepcopy 

from src.synthetic import GaussianDataset
from src.data import get_retain_forget_datasets, get_dataloaders
from src.train import train, L2RegularizedCrossEntropyLoss
from src.eval import evaluate
from src.forget import forget, calculate_hessian, calculate_grad, calculate_update, update_model
from src.utils import get_module_device
from tqdm import tqdm

In [2]:
num_classes = 10
num_samples = 15000
dim = 100
mean = np.zeros(dim)
cov = np.eye(dim)
## surr cov
surr_cov = np.eye(dim)
surr_cov[0, -1] = 0.5
surr_cov[-1, 0] = 0.5
##
dataset = GaussianDataset(num_samples, num_classes, mean, cov)
surr_dataset = dataset.create_surr(mean, surr_cov)
train_dataset, test_dataset = get_retain_forget_datasets(dataset, 0.2)

kl_distance = surr_dataset.calculate_kl_between(dataset)
print('given kl distance:{}'.format(kl_distance))

eps = 5 * (math.e ** 3)
delta = 1

given kl distance:0.14384103622589045


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = L2RegularizedCrossEntropyLoss(l2_lambda=0.1)

In [4]:
def print_eval(model_arg):
    print('#######################################')
    print('train:')
    evaluate(train_loader, model_arg, criterion, device=device)
    print('#######################################')
    print('#######################################')
    print('val:')
    evaluate(val_loader, model_arg, criterion, device=device)
    print('#######################################')
    print('#######################################')
    print('retain:')
    evaluate(retain_loader, model_arg, criterion, device=device)
    print('#######################################')
    print('#######################################')
    print('forget:')
    evaluate(forget_loader, model_arg, criterion, device=device)
    print('#######################################')
    print('#######################################')
    print('surrogate:')
    evaluate(surr_loader, model_arg, criterion, device=device)
    print('#######################################')

In [5]:
retain_dataset, forget_dataset = get_retain_forget_datasets(train_dataset, 0.1)
train_loader, val_loader = get_dataloaders([train_dataset, test_dataset], batch_size=256)
retain_loader = get_dataloaders(retain_dataset, batch_size=256)
forget_loader = get_dataloaders(forget_dataset, batch_size=256)
surr_loader = get_dataloaders(surr_dataset, batch_size=256)
model = nn.Linear(dim, num_classes, bias=False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(train_loader, val_loader, model, criterion, optimizer, num_epoch=300, device=device)
print_eval(model)
model = model.to('cpu')

eval: 100%|██████████| 12/12 [00:00<00:00, 395.16batch/s, acc=0.559, loss=2.06]


#######################################
train:


eval: 100%|██████████| 47/47 [00:00<00:00, 389.76batch/s, acc=0.644, loss=2.05]


#######################################
#######################################
val:


eval: 100%|██████████| 12/12 [00:00<00:00, 372.98batch/s, acc=0.559, loss=2.09]


...

#######################################
#######################################
forget:


eval: 100%|██████████| 5/5 [00:00<00:00, 479.40batch/s, acc=0.654, loss=2.01]


#######################################
#######################################
surrogate:


eval: 100%|██████████| 59/59 [00:00<00:00, 457.04batch/s, acc=0.589, loss=2.04]

#######################################


In [6]:
# retrain from scratch
rmodel = nn.Linear(dim, num_classes, bias=False).to(device)
optimizer = torch.optim.Adam(rmodel.parameters(), lr=0.001)
train(retain_loader, val_loader, rmodel, criterion, optimizer, num_epoch=300, device=device)

print_eval(rmodel)
rmodel = rmodel.to('cpu')

eval: 100%|██████████| 12/12 [00:00<00:00, 356.62batch/s, acc=0.552, loss=2.04]


#######################################
train:


eval: 100%|██████████| 47/47 [00:00<00:00, 360.95batch/s, acc=0.635, loss=2.03]


#######################################
#######################################
val:


eval: 100%|██████████| 12/12 [00:00<00:00, 430.46batch/s, acc=0.552, loss=2.06]


...

#######################################
#######################################
forget:


eval: 100%|██████████| 5/5 [00:00<00:00, 417.96batch/s, acc=0.572, loss=2.07]


#######################################
#######################################
surrogate:


eval: 100%|██████████| 59/59 [00:00<00:00, 449.80batch/s, acc=0.593, loss=2.05]

#######################################


In [7]:
# retrain from scratch
surrmodel = nn.Linear(dim, num_classes, bias=False).to(device)
optimizer = torch.optim.Adam(surrmodel.parameters(), lr=0.001)
train(surr_loader, val_loader, surrmodel, criterion, optimizer, num_epoch=300, device=device)

print_eval(surrmodel)
surrmodel = surrmodel.to('cpu')

eval: 100%|██████████| 12/12 [00:00<00:00, 403.96batch/s, acc=0.531, loss=2.08]


#######################################
train:


eval: 100%|██████████| 47/47 [00:00<00:00, 398.61batch/s, acc=0.525, loss=2.05]


#######################################
#######################################
val:


eval: 100%|██████████| 12/12 [00:00<00:00, 325.71batch/s, acc=0.531, loss=2.04]


...

#######################################
#######################################
forget:


eval: 100%|██████████| 5/5 [00:00<00:00, 536.91batch/s, acc=0.51, loss=2.08]


#######################################
#######################################
surrogate:


eval: 100%|██████████| 59/59 [00:00<00:00, 435.52batch/s, acc=0.61, loss=2.06] 


#######################################


In [8]:
# forget with exact
model = model.to(device)
# fmodel = forget(model, train_loader, forget_loader, forget_loader, criterion, save_path='tmp', eps=eps, delta=delta)
fmodel = forget(model, train_loader, forget_loader, forget_loader, criterion, save_path='tmp')
model = model.to('cpu')
print_eval(fmodel)
fmodel = fmodel.to('cpu')

Calculating Hessian: 100%|██████████| 5/5 [00:36<00:00,  7.35s/it]


#######################################
train:


eval: 100%|██████████| 47/47 [00:00<00:00, 561.34batch/s, acc=0.616, loss=2.03]


#######################################
#######################################
val:


eval: 100%|██████████| 12/12 [00:00<00:00, 558.75batch/s, acc=0.537, loss=2.03]


...

#######################################
#######################################
forget:


eval: 100%|██████████| 5/5 [00:00<00:00, 554.16batch/s, acc=0.56, loss=2.09]


#######################################
#######################################
surrogate:


eval: 100%|██████████| 59/59 [00:00<00:00, 555.95batch/s, acc=0.575, loss=2.07]


#######################################


In [9]:
# forget with surrogate
# forget with exact
model = model.to(device)
surrmodel = surrmodel.to(device)
smodel = forget(model, surr_loader, forget_loader, forget_loader, criterion, linear=True, num_class=num_classes, eps=eps, delta=delta, surr=True, known=True, surr_loader=surr_loader, surr_model=surrmodel, kl_distance=kl_distance)
model = model.to('cpu')
print_eval(smodel)
smodel = smodel.to('cpu')

TypeError: forget() got an unexpected keyword argument 'linear'